# `multifunctional`

`multifunctional` is a library for handling multifunctional activities in the Brightway LCA software framework.

In [2]:
import multifunctional as mf
import bw2data as bd
import bw2io as bi
import bw2calc as bc

/Users/cmutel/Code/Brightway/brightway2-calc/bw2calc/__init__.py:45: UserWarning: 
It seems like you have an ARM architecture, but haven't installed scikit-umfpack:

    https://pypi.org/project/scikit-umfpack/

Installing it could give you much faster calculations.

  warnings.warn(UMFPACK_WARNING)


In [3]:
bi.remote.install_project(
    'ecoinvent-3.10-biosphere', 
    project_name="multifunctional demo", 
    overwrite_existing=True
)

Restoring project backup archive - this could take a few minutes...
Restored project: multifunctional demo


'multifunctional demo'

In [4]:
bd.projects.set_current("multifunctional demo")

Applying automatic update: 4.0 database search directories
Reindexing database ecoinvent-3.10-biosphere


In [5]:
co2 = bd.get_node(
    name='Carbon dioxide, fossil',
    categories=('air', 'non-urban air or from high stacks'),
)

/Users/cmutel/Code/Brightway/brightway2-data/bw2data/utils.py:377: UserWarning: Given search criteria very broad; try to specify at least a database
  warnings.warn("Given search criteria very broad; try to specify at least a database")


In [6]:
mf_db = mf.MultifunctionalDatabase("emojis FTW")
mf_db.register(default_allocation="price")

In [7]:
mf_data = {
    ("emojis FTW", "😼"): {
        "type": "product",
        "name": "meow",
        "unit": "kg",
    },
    ("emojis FTW", "🐶"): {
        "type": "product",
        "name": "woof",
        "unit": "kg",
    },
    ("emojis FTW", "1"): {
        "name": "process - 1",
        "location": "somewhere",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "😼"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                },
            },
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "🐶"),
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                },
            },
            {
                "functional": False,
                "type": "biosphere",
                "input": co2.key,
                "amount": 100
            }
        ],
    }
}

In [8]:
mf_db.write(mf_data)

100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 5067.62it/s]

Vacuuming database 


In [9]:
for node in mf_db:
    print(node, node.id)
    for edge in node.edges():
        print("\t", edge)

'woof' (kg, None, None) 4364
Multifunctional: 'process - 1' (None, somewhere, None) 4365
	 Exchange: 4 kg 'meow' (kg, None, None) to Multifunctional: 'process - 1' (None, somewhere, None)>
	 Exchange: 6 kg 'woof' (kg, None, None) to Multifunctional: 'process - 1' (None, somewhere, None)>
	 Exchange: 100 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')) to Multifunctional: 'process - 1' (None, somewhere, None)>
'meow' (kg, None, None) 4363
Read-only allocated process: 'process - 1' (kg, somewhere, None) 4366
	 Exchange: 4 kg 'meow' (kg, None, None) to Read-only allocated process: 'process - 1' (kg, somewhere, None)>
	 Exchange: 28.000000000000004 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')) to Read-only allocated process: 'process - 1' (kg, somewhere, None)>
Read-only allocated process: 'process - 1' (kg, somewhere, None) 4367
	 Exchange: 6 kg 'woof' (kg, None, None) to Read-only allocated

In [10]:
IPCC = (
    'IPCC 2021',
    'climate change',
    'global temperature change potential (GTP100)'
)

In [11]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

21.987000000000002

In [14]:
mf_db.metadata["default_allocation"] = "mass"
mf_db.process()

In [15]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

39.2625

# Custom allocation based on new properties

You need to tell the system to create a function for each new property

In [17]:
mf.allocation_strategies['happiness'] = mf.property_allocation(property_label='😊')

In [19]:
happy = {
    ("emojis FTW", "😼"): {
        "type": "product",
        "name": "meow",
        "unit": "kg",
    },
    ("emojis FTW", "🐶"): {
        "type": "product",
        "name": "woof",
        "unit": "kg",
    },
    ("emojis FTW", "1"): {
        "name": "process - 1",
        "location": "somewhere",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "😼"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                    "😊": 10,
                },
            },
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "🐶"),
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                },
            },
            {
                "functional": False,
                "type": "biosphere",
                "input": co2.key,
                "amount": 100
            }
        ],
    }
}
mf_db.write(happy)

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 16299.11it/s]

Vacuuming database 


We can check to make sure that the properties we want are available:

In [20]:
mf.list_available_properties("emojis FTW")

[('price',
  <MessageType.ALL_VALID: 'All properties found and have correct type'>),
 ('😊', <MessageType.MISSING_PROPERTY: 'Missing property'>),
 ('mass',
  <MessageType.ALL_VALID: 'All properties found and have correct type'>)]

In [21]:
happy_dog = {
    ("emojis FTW", "😼"): {
        "type": "product",
        "name": "meow",
        "unit": "kg",
    },
    ("emojis FTW", "🐶"): {
        "type": "product",
        "name": "woof",
        "unit": "kg",
    },
    ("emojis FTW", "1"): {
        "name": "process - 1",
        "location": "somewhere",
        "exchanges": [
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "😼"),
                "amount": 4,
                "properties": {
                    "price": 7,
                    "mass": 6,
                    "😊": 2,
                },
            },
            {
                "functional": True,
                "type": "production",
                "input": ("emojis FTW", "🐶"),
                "amount": 6,
                "properties": {
                    "price": 12,
                    "mass": 4,
                    "😊": 50,
                },
            },
            {
                "functional": False,
                "type": "biosphere",
                "input": co2.key,
                "amount": 100
            }
        ],
    }
}
mf_db.write(happy_dog)

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 17747.41it/s]

Vacuuming database 


In [24]:
mf_db.metadata["default_allocation"] = "happiness"
mf_db.process()

In [25]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

2.0396103701591493

# Writing custom allocation functions

In [28]:
from functools import partial

def allocation_factor(edge_data: dict, node: mf.MaybeMultifunctionalProcess) -> float:
    """Nonsensical allocation factor generation"""
    if edge_data['input'] == ('emojis FTW', '😼'):
        return 7
    else:
        return 17

mf.allocation_strategies['silly'] = partial(
    mf.generic_allocation,
    func=allocation_factor,
    strategy_label="something silly"
)

In [29]:
mf_db.metadata["default_allocation"] = "silly"
mf_db.process()

In [30]:
lca = bc.LCA({bd.get_node(name="meow"): 3.141}, IPCC)
lca.lci()
lca.lcia()
lca.score

22.903124500751495